## Imports

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
from sklearn.preprocessing import MinMaxScaler

## Importing Data

##### Custom Dataset Class

In [8]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, file, scaler):
    self.df = pd.read_csv(file)
    self.sc = scaler
    self.sc.fit(self.df)

  def __len__(self):
    return self.df.shape[0]
  
  def __getitem__(self, idx):
    raw = self.df.iloc[idx].values
    if type(idx) == int:
      raw = raw.reshape(1, -1)
    raw = self.sc.transform(raw)
    data = torch.tensor(raw[:, 1:], dtype=torch.float32) # Features
    label = torch.tensor(raw[:, 0], dtype=torch.float32) # Label
    return data, label

##### Importing Training Data and Scaling based on each

In [9]:
""" Center Surround Training Data """
center_surround_train_df = pd.read_csv("./center_surround_train.csv")
center_surround_scaler = MinMaxScaler().fit(center_surround_train_df)
center_surround_train_data = CustomDataset('./center_surround_train.csv', center_surround_scaler)
center_surround_test_data = CustomDataset('./center_surround_test.csv', center_surround_scaler)
center_surround_valid_data = CustomDataset('./center_surround_valid.csv', center_surround_scaler)

center_surround_train_loader = DataLoader(center_surround_train_data, batch_size=64, shuffle=True)
center_surround_test_loader = DataLoader(center_surround_test_data, batch_size=64, shuffle=True)

""" Spiral Test Training Data """
spiral_train_df = pd.read_csv("./spiral_train.csv")
spiral_scaler = MinMaxScaler().fit(spiral_train_df)
spiral_train_data = CustomDataset('./spiral_train.csv', spiral_scaler)
spiral_test_data = CustomDataset('./spiral_test.csv', spiral_scaler)
spiral_valid_data = CustomDataset('./spiral_valid.csv', spiral_scaler)

""" Two Gaussians Training Data """
two_gaussians_train_df = pd.read_csv("./two_gaussians_train.csv")
two_gaussians_scaler = MinMaxScaler().fit(two_gaussians_train_df)
two_gaussians_train_data = CustomDataset('./two_gaussians_train.csv', two_gaussians_scaler)
two_gaussians_test_data = CustomDataset('./two_gaussians_test.csv', two_gaussians_scaler)
two_gaussians_valid_data = CustomDataset('./two_gaussians_valid.csv', two_gaussians_scaler)

""" XORTraining Data """
xor_train_df = pd.read_csv("./xor_train.csv")
xor_scaler = MinMaxScaler().fit(xor_train_df)
xor_train_data = CustomDataset('./xor_train.csv', xor_scaler)
xor_test_data = CustomDataset('./xor_test.csv', xor_scaler)
xor_valid_data = CustomDataset('./xor_valid.csv', xor_scaler)

In [15]:
for X, y in center_surround_train_loader:
  for i in range(3):  
    print("Example #: ", i + 1)
    print("Features: ", X[i])
    print("Label: ", y[i])
  break  

Example #:  1
Features:  tensor([[0.3893, 0.4663]])
Label:  tensor([0.])
Example #:  2
Features:  tensor([[0.7732, 0.5471]])
Label:  tensor([1.])
Example #:  3
Features:  tensor([[0.1260, 0.4609]])
Label:  tensor([1.])


/Users/pmcslarrow/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/pmcslarrow/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/pmcslarrow/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/pmcslarrow/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/pmcslarrow/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/pmcslarr